In [104]:
import numpy as np
import pandas as pd
import random


class fwa:
    """
    Firework Algorithm (FWA)
    """
    def __init__(self, n, function, lb, ub, dimension, iteration, m1=7, m2=7, eps=0.001, amp=2, a=0.3, b=3):
        """
        :param n: number of fireworks
        :param function: objective function
        :param lb: lower bound
        :param ub: upper bound
        :param dimension: dimensionality of the problem
        :param iteration: number of iterations
        :param m1: parameter controlling the number of normal sparks
        :param m2: parameter controlling the number of Gaussian sparks
        :param eps: small constant to avoid division by zero
        :param amp: amplitude of the explosion
        :param a: lower bound for the number of sparks
        :param b: upper bound for the number of sparks
        """
        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        self._points = [self.__agents]

        Pbest = self.__agents[
            np.array([function(x) for x in self.__agents]).argmin()
        ]
        Gbest = Pbest

        for i in range(iteration):
            Ymin = function(Pbest)
            Ymax = max([function(x) for x in self.__agents])
            sparks = []
            for fw in self.__agents:
                self.__explosion_operator(sparks, fw, function, dimension, m1, eps, amp, Ymin, Ymax, a, b)
                self.__gaussian_mutation(sparks, fw, dimension, m2)

            self.__mapping_rule(sparks, lb, ub, dimension)
            self.__selection(sparks, n, function)
            self._points.append(self.__agents)

            Pbest = self.__agents[
                np.array([function(x) for x in self.__agents]).argmin()
            ]
            if function(Pbest) < function(Gbest):
                Gbest = Pbest

        self.Gbest = Gbest
        self.Gbest_value = function(Gbest)  # Adicionando o valor da função objetivo para Gbest

    def __explosion_operator(self, sparks, fw, function, dimension, m, eps, amp, Ymin, Ymax, a, b):
        sparks_num = self.__round(
            m * (Ymax - function(fw) + eps) /
            (sum([Ymax - function(fwk) for fwk in self.__agents]) + eps),
            m, a, b
        )

        amplitude = amp * (function(fw) - Ymin + eps) / \
                    (sum([function(fwk) - Ymin for fwk in self.__agents]) + eps)

        for j in range(int(sparks_num)):
            sparks.append(np.array(fw))
            for k in range(dimension):
                if random.choice([True, False]):
                    sparks[-1][k] += random.uniform(-amplitude, amplitude)

    def __gaussian_mutation(self, sparks, fw, dimension, m):
        for j in range(m):
            g = np.random.normal(1, 1)
            sparks.append(np.array(fw))
            for k in range(dimension):
                if random.choice([True, False]):
                    sparks[-1][k] *= g

    def __mapping_rule(self, sparks, lb, ub, dimension):
        for i in range(len(sparks)):
            for j in range(dimension):
                if sparks[i][j] > ub or sparks[i][j] < lb:
                    sparks[i][j] = lb + (sparks[i][j] - lb) % (ub - lb)

    def __selection(self, sparks, n, function):
        self.__agents = sorted(np.concatenate((self.__agents, sparks)), key=function)[:n]

    def __round(self, s, m, a, b):
        if s < a * m:
            return round(a * m)
        elif s > b * m:
            return round(b * m)
        else:
            return round(s)


# Define a função de Ackley
def ackley_function(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    d = len(x)

    term1 = -a * np.exp(-b * np.sqrt(np.sum(np.square(x)) / d))
    term2 = -np.exp(np.sum(np.cos(c * x)) / d)

    return term1 + term2 + a + np.exp(1)


# Parâmetros
num_fireworks = 5  # Número de fogos
lb = -5             # Limite inferior
ub = 5              # Limite superior
dimension = 3       # Dimensão do problema
iterations = 50    # Iterações máximas
num_executions = 20  # Número de execuções do algoritmo

# Executar o algoritmo FWA 20 vezes
results = []
for i in range(num_executions):
    fwa_solver = fwa(
        n=num_fireworks,
        function=ackley_function,
        lb=lb,
        ub=ub,
        dimension=dimension,
        iteration=iterations
    )
    results.append(fwa_solver.Gbest_value)

# Calcular a média dos resultados
mean_result = np.mean(results)

# Criar um DataFrame para exportar os resultados
results_df = pd.DataFrame({
    "Execução": [f"Execução {i+1}" for i in range(num_executions)],
    "Melhor Valor": [f"{value:.4e}" for value in results]
})

# Adicionar a média como última linha
results_df.loc[len(results_df)] = ["Média", f"{mean_result:.4e}"]

# Exportar para um arquivo Excel
excel_file = "resultados_fwa.xlsx"
results_df.to_excel(excel_file, index=False)

print(f"Resultados salvos no arquivo Excel: {excel_file}")
print(f"Média dos melhores valores: {mean_result:.4e}")


Resultados salvos no arquivo Excel: resultados_fwa.xlsx
Média dos melhores valores: 4.4409e-16


In [132]:
import numpy as np
import pandas as pd


class PSO:
    """
    Particle Swarm Optimization
    """

    def __init__(self, n, function, lb, ub, dimension, iteration, w=0.5, c1=1, c2=1):
        """
        :param n: number of particles (agents)
        :param function: objective function
        :param lb: lower bound for search space
        :param ub: upper bound for search space
        :param dimension: dimensionality of the problem
        :param iteration: number of iterations
        :param w: inertia weight
        :param c1: cognitive component weight (personal influence)
        :param c2: social component weight (global influence)
        """
        # Initialize particle positions and velocities
        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        velocity = np.zeros((n, dimension))  # Initial velocities set to 0

        # Initialize personal best (Pbest) and global best (Gbest)
        Pbest = self.__agents.copy()  # Personal best positions
        Pbest_values = np.array([function(x) for x in Pbest])  # Personal best fitness values
        Gbest = Pbest[np.argmin(Pbest_values)]  # Global best position
        Gbest_value = np.min(Pbest_values)  # Global best fitness value

        # Optimization loop
        for _ in range(iteration):
            # Update particle velocities and positions
            r1 = np.random.random((n, dimension))  # Random values for cognitive component
            r2 = np.random.random((n, dimension))  # Random values for social component

            # Update velocity
            velocity = w * velocity + c1 * r1 * (Pbest - self.__agents) + c2 * r2 * (Gbest - self.__agents)
            
            # Update position
            self.__agents += velocity
            
            # Ensure particles stay within bounds
            self.__agents = np.clip(self.__agents, lb, ub)

            # Evaluate fitness at new positions
            fitness_values = np.array([function(x) for x in self.__agents])

            # Update personal bests (Pbest)
            for i in range(n):
                if fitness_values[i] < Pbest_values[i]:
                    Pbest[i] = self.__agents[i]
                    Pbest_values[i] = fitness_values[i]

            # Update global best (Gbest)
            if fitness_values.min() < Gbest_value:
                Gbest = self.__agents[np.argmin(fitness_values)]
                Gbest_value = fitness_values.min()

        # Store the results
        self.Gbest = Gbest
        self.Gbest_value = Gbest_value

    def get_best_solution(self):
        """Return the best solution found and its fitness value."""
        return self.Gbest, self.Gbest_value


# Define a função de Ackley
def ackley_function(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    d = len(x)

    term1 = -a * np.exp(-b * np.sqrt(np.sum(np.square(x)) / d))
    term2 = -np.exp(np.sum(np.cos(c * x)) / d)

    return term1 + term2 + a + np.exp(1)


# Parâmetros do PSO
num_particles = 10  # Número de partículas
lb = -5             # Limite inferior
ub = 5              # Limite superior
dimension = 3       # Dimensão do problema
iterations = 100    # Iterações máximas
num_executions = 20  # Número de execuções do algoritmo

# Executar o algoritmo PSO 20 vezes
results = []
for i in range(num_executions):
    pso_solver = PSO(
        n=num_particles,
        function=ackley_function,
        lb=lb,
        ub=ub,
        dimension=dimension,
        iteration=iterations,
        w=0.4,
        c1=1,
        c2=2
    )
    results.append(pso_solver.Gbest_value)

# Calcular a média dos resultados
mean_result = np.mean(results)

# Criar um DataFrame para exportar os resultados
results_df = pd.DataFrame({
    "Execução": [f"Execução {i+1}" for i in range(num_executions)],
    "Melhor Valor": [f"{value:.4e}" for value in results]
})

# Adicionar a média como última linha
results_df.loc[len(results_df)] = ["Média", f"{mean_result:.4e}"]

# Exportar para um arquivo Excel
excel_file = "resultados_pso.xlsx"
results_df.to_excel(excel_file, index=False)

print(f"Resultados salvos no arquivo Excel: {excel_file}")
print(f"Média dos melhores valores: {mean_result:.4e}")


Resultados salvos no arquivo Excel: resultados_pso.xlsx
Média dos melhores valores: 3.8868e-01


In [58]:
import numpy as np
import pandas as pd

# Método de execução múltipla
def run_multiple_executions(algorithm_class, function, lb, ub, dimension, iteration, num_executions, n, **kwargs):
    best_values = []
    
    for _ in range(num_executions):
        # Instancia o algoritmo e executa
        solver = algorithm_class(
            n=n,  # Número de fogos de artifício ou partículas
            function=function,
            lb=lb,
            ub=ub,
            dimension=dimension,
            iteration=iteration,
            **kwargs
        )
        # Salva o melhor valor encontrado na execução
        best_values.append(solver.Gbest_value)
    
    # Calcula estatísticas
    mean_value = np.mean(best_values)
    std_dev = np.std(best_values)

    return mean_value, std_dev, best_values

# Parâmetros gerais
lb, ub = -5, 5
dimension = 2
iteration = 100
num_executions = 20

# Executar FWA
mean_fwa, std_fwa, all_fwa_values = run_multiple_executions(
    algorithm_class=fwa,
    function=ackley_function,
    lb=lb,
    ub=ub,
    dimension=dimension,
    iteration=iteration,
    num_executions=num_executions,
    n=10,  # Número de fogos de artifício
    m1=7,
    m2=7,
    amp=2
)

# Executar PSO
mean_pso, std_pso, all_pso_values = run_multiple_executions(
    algorithm_class=PSO,
    function=ackley_function,
    lb=lb,
    ub=ub,
    dimension=dimension,
    iteration=iteration,
    num_executions=num_executions,
    n=10,  # Número de partículas
    w=0.5,
    c1=1.5,
    c2=1.5
)

# Criar um DataFrame para os resultados detalhados por execução
fwa_results_df = pd.DataFrame({
    "Execução": [f"Execução {i+1}" for i in range(len(all_fwa_values))],
    "Melhor Valor FWA": [f"{value:.4e}" for value in all_fwa_values]
})

pso_results_df = pd.DataFrame({
    "Execução": [f"Execução {i+1}" for i in range(len(all_pso_values))],
    "Melhor Valor PSO": [f"{value:.4e}" for value in all_pso_values]
})

# Criar um DataFrame para as métricas gerais
metrics_df = pd.DataFrame({
    "Métrica": ["Média (Melhor Valor)", "Desvio Padrão"],
    "FWA": [f"{mean_fwa:.4e}", f"{std_fwa:.4e}"],
    "PSO": [f"{mean_pso:.4e}", f"{std_pso:.4e}"]
})

# Exportar para Excel em abas diferentes
excel_file = "resultados_algoritmos_detalhados.xlsx"
with pd.ExcelWriter(excel_file) as writer:
    fwa_results_df.to_excel(writer, sheet_name="Resultados FWA", index=False)
    pso_results_df.to_excel(writer, sheet_name="Resultados PSO", index=False)
    metrics_df.to_excel(writer, sheet_name="Métricas Gerais", index=False)

print(f"Resultados salvos no arquivo Excel: {excel_file}")


Resultados salvos no arquivo Excel: resultados_algoritmos_detalhados.xlsx


In [69]:
import numpy as np
import pandas as pd
import random


class fwa:
    """
    Firework Algorithm (FWA)
    """
    def __init__(self, n, function, lb, ub, dimension, iteration, m1=7, m2=7, eps=0.001, amp=2, a=0.3, b=3):
        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        self._points = [self.__agents]

        Pbest = self.__agents[
            np.array([function(x) for x in self.__agents]).argmin()
        ]
        Gbest = Pbest

        for i in range(iteration):
            Ymin = function(Pbest)
            Ymax = max([function(x) for x in self.__agents])
            sparks = []
            for fw in self.__agents:
                self.__explosion_operator(sparks, fw, function, dimension, m1, eps, amp, Ymin, Ymax, a, b)
                self.__gaussian_mutation(sparks, fw, dimension, m2)

            self.__mapping_rule(sparks, lb, ub, dimension)
            self.__selection(sparks, n, function)
            self._points.append(self.__agents)

            Pbest = self.__agents[
                np.array([function(x) for x in self.__agents]).argmin()
            ]
            if function(Pbest) < function(Gbest):
                Gbest = Pbest

        self.Gbest = Gbest
        self.Gbest_value = function(Gbest)

    def __explosion_operator(self, sparks, fw, function, dimension, m, eps, amp, Ymin, Ymax, a, b):
        sparks_num = self.__round(
            m * (Ymax - function(fw) + eps) /
            (sum([Ymax - function(fwk) for fwk in self.__agents]) + eps),
            m, a, b
        )

        amplitude = amp * (function(fw) - Ymin + eps) / \
                    (sum([function(fwk) - Ymin for fwk in self.__agents]) + eps)

        for j in range(int(sparks_num)):
            sparks.append(np.array(fw))
            for k in range(dimension):
                if random.choice([True, False]):
                    sparks[-1][k] += random.uniform(-amplitude, amplitude)

    def __gaussian_mutation(self, sparks, fw, dimension, m):
        for j in range(m):
            g = np.random.normal(1, 1)
            sparks.append(np.array(fw))
            for k in range(dimension):
                if random.choice([True, False]):
                    sparks[-1][k] *= g

    def __mapping_rule(self, sparks, lb, ub, dimension):
        for i in range(len(sparks)):
            for j in range(dimension):
                if sparks[i][j] > ub or sparks[i][j] < lb:
                    sparks[i][j] = lb + (sparks[i][j] - lb) % (ub - lb)

    def __selection(self, sparks, n, function):
        self.__agents = sorted(np.concatenate((self.__agents, sparks)), key=function)[:n]

    def __round(self, s, m, a, b):
        if s < a * m:
            return round(a * m)
        elif s > b * m:
            return round(b * m)
        else:
            return round(s)


class PSO:
    """
    Particle Swarm Optimization
    """
    def __init__(self, n, function, lb, ub, dimension, iteration, w=0.5, c1=1, c2=1):
        self.__agents = np.random.uniform(lb, ub, (n, dimension))
        velocity = np.zeros((n, dimension))

        Pbest = self.__agents.copy()
        Pbest_values = np.array([function(x) for x in Pbest])
        Gbest = Pbest[np.argmin(Pbest_values)]
        Gbest_value = np.min(Pbest_values)

        for _ in range(iteration):
            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))

            velocity = w * velocity + c1 * r1 * (Pbest - self.__agents) + c2 * r2 * (Gbest - self.__agents)
            self.__agents += velocity
            self.__agents = np.clip(self.__agents, lb, ub)

            fitness_values = np.array([function(x) for x in self.__agents])

            for i in range(n):
                if fitness_values[i] < Pbest_values[i]:
                    Pbest[i] = self.__agents[i]
                    Pbest_values[i] = fitness_values[i]

            if fitness_values.min() < Gbest_value:
                Gbest = self.__agents[np.argmin(fitness_values)]
                Gbest_value = fitness_values.min()

        self.Gbest = Gbest
        self.Gbest_value = Gbest_value


def ackley_function(x):
    a = 20
    b = 0.2
    c = 2 * np.pi
    d = len(x)

    term1 = -a * np.exp(-b * np.sqrt(np.sum(np.square(x)) / d))
    term2 = -np.exp(np.sum(np.cos(c * x)) / d)

    return term1 + term2 + a + np.exp(1)


# Configurações
num_particles = 10
lb = -5
ub = 5
iterations = 100
num_executions = 100
dimensions = [2, 3]

# Resultados
results = []

for dimension in dimensions:
    # FWA
    fwa_results = []
    for _ in range(num_executions):
        fwa_solver = fwa(
            n=num_particles,
            function=ackley_function,
            lb=lb,
            ub=ub,
            dimension=dimension,
            iteration=iterations
        )
        fwa_results.append(fwa_solver.Gbest_value)
    
    # PSO
    pso_results = []
    for _ in range(num_executions):
        pso_solver = PSO(
            n=num_particles,
            function=ackley_function,
            lb=lb,
            ub=ub,
            dimension=dimension,
            iteration=iterations,
            w=0.5,
            c1=1.5,
            c2=1.5
        )
        pso_results.append(pso_solver.Gbest_value)

    results.append({
        "Dimensão": dimension,
        "Média FWA": np.mean(fwa_results),
        "Desvio Padrão FWA": np.std(fwa_results),
        "Média PSO": np.mean(pso_results),
        "Desvio Padrão PSO": np.std(pso_results)
    })

# Criar DataFrame
results_df = pd.DataFrame(results)

# Exportar para Excel
excel_file = "resultados_fwa_pso_ackley.xlsx"
results_df.to_excel(excel_file, index=False)

print(f"Os resultados foram salvos no arquivo Excel: {excel_file}")


Os resultados foram salvos no arquivo Excel: resultados_fwa_pso_ackley.xlsx
